<a href="https://colab.research.google.com/github/neevshah1273/Sarcasm-Detector/blob/master/Sarcasm_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
!pip3 install pandas
!pip3 install sklearn
!pip3 install tensorflow

In [97]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow import keras

In [98]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [99]:
df=pd.read_json('/content/drive/My Drive/Colab Notebooks/Sarcasm_Headlines_Dataset_v2.json', lines = True)
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  28619 non-null  int64 
 1   headline      28619 non-null  object
 2   article_link  28619 non-null  object
dtypes: int64(1), object(2)
memory usage: 670.9+ KB


In [101]:
df['is_sarcastic'].value_counts()

0    14985
1    13634
Name: is_sarcastic, dtype: int64

In [102]:
sentences = df['headline']
labels = df['is_sarcastic']

train_sentences, val_sentences, train_labels, val_labels = train_test_split(sentences, labels, test_size=0.2, random_state=0)

print(train_sentences.shape)
print(val_sentences.shape)
print(train_labels.shape)
print(val_labels.shape)

(22895,)
(5724,)
(22895,)
(5724,)


In [103]:
vocab_size = 10000
oov_token = '<00V>'
max_length = 100
padding_type = 'post'
trunc_type = 'post'


tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

val_sequences = tokenizer.texts_to_sequences(val_sentences)
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [104]:
embedding_dim = 16
num_epochs = 10
batch_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']
             )

history = model.fit(train_padded, train_labels, batch_size=batch_size, epochs=num_epochs, 
                    verbose=2)

Epoch 1/10
229/229 - 2s - loss: 0.6889 - accuracy: 0.5225
Epoch 2/10
229/229 - 1s - loss: 0.6235 - accuracy: 0.7316
Epoch 3/10
229/229 - 1s - loss: 0.4443 - accuracy: 0.8402
Epoch 4/10
229/229 - 1s - loss: 0.3396 - accuracy: 0.8699
Epoch 5/10
229/229 - 1s - loss: 0.2914 - accuracy: 0.8880
Epoch 6/10
229/229 - 1s - loss: 0.2601 - accuracy: 0.8996
Epoch 7/10
229/229 - 1s - loss: 0.2371 - accuracy: 0.9087
Epoch 8/10
229/229 - 1s - loss: 0.2163 - accuracy: 0.9168
Epoch 9/10
229/229 - 1s - loss: 0.2008 - accuracy: 0.9238
Epoch 10/10
229/229 - 1s - loss: 0.1862 - accuracy: 0.9311


In [105]:
val_loss, val_accuracy = model.evaluate(val_padded, val_labels)
print('Val loss: {}, Val accuracy: {}'.format(val_loss, val_accuracy*100))

quick_test_sentence = [
    'breathing oxygen linked to staying alive',
    'india wins the cup',
    'Cows lose their jobs as milk prices drop'
    
]

quick_test_sequences = tokenizer.texts_to_sequences(quick_test_sentence)
quick_test_padded = pad_sequences(quick_test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
quick_test_sentiments = model.predict(quick_test_padded)

for i in range(len(quick_test_sentiments)):
    print('\n' + quick_test_sentence[i])
    if 0 < quick_test_sentiments[i] < .50:
        print('Unlikely sarcasm. Sarcasm score: {}'.format(quick_test_sentiments[i]*100))
    elif .50 < quick_test_sentiments[i] < .75:
        print('Possible sarcasm. Sarcasm score: {}'.format(quick_test_sentiments[i]*100))
    elif .75 >  quick_test_sentiments[i] < 1:
        print('Sarcasm. Sarcasm score:  {}'.format(quick_test_sentiments[i]*100))
    else:
        print('Not in range')

179/179 [==============================] - 0s 1ms/step - loss: 0.3409 - accuracy: 0.8520
Val loss: 0.3409145474433899, Val accuracy: 85.20265817642212

breathing oxygen linked to staying alive
Possible sarcasm. Sarcasm score: [55.201286]

india wins the cup
Unlikely sarcasm. Sarcasm score: [6.0227394]

Cows lose their jobs as milk prices drop
Possible sarcasm. Sarcasm score: [57.380558]
